# Get Repositories

This script will get all GitHub repositories that you own, collaborate on, or own the organization of. Just update your `gh_token` with your GitHub token and run the script. Make a token here: https://github.com/settings/tokens


In [ ]:
import os
import subprocess
import requests
from concurrent.futures import ThreadPoolExecutor

# Your GitHub Personal Access Token
gh_token = "Your Github Personal Access Token"

# Directory to clone repositories
repo_dir = "C:/Repos/"


# Function to fetch repositories
def fetch_repos(url, paginated=True):
    all_repos = []
    if paginated == False:
        response = requests.get(
            url,
            headers={
                "Authorization": f"token {gh_token}",
                "Accept": "application/vnd.github.v3+json",
            },
        )
        return response.json()
    page = 1
    while True:
        response = requests.get(
            f"{url}&page={page}",
            headers={
                "Authorization": f"token {gh_token}",
                "Accept": "application/vnd.github.v3+json",
            },
        )
        repos = response.json()
        if not repos:
            break
        all_repos.extend(repos)
        page += 1
    return all_repos


def get_all_repos():
    # Fetch personal and collaborated repositories
    user_repos = fetch_repos(f"https://api.github.com/user/repos?type=all")
    print(f"Found {len(user_repos)} user repositories")
    # Fetch organizations
    orgs = fetch_repos(f"https://api.github.com/user/orgs", False)
    print(f"Found {len(orgs)} organizations associated with user")

    # Fetch repositories from organizations
    org_repos = []
    for org in orgs:
        org_name = org["login"]
        org_repos += fetch_repos(f"https://api.github.com/orgs/{org_name}/repos", False)
    print(f"Found {len(org_repos)} repositories from organizations")

    # Combine all repositories, avoiding duplicates
    all_repos = []
    for repo in user_repos + org_repos:
        if repo not in all_repos:
            all_repos.append(repo)
    return all_repos


def get_repository(repo):
    repo_name = repo["full_name"]
    clone_url = repo["clone_url"]
    dir_path = os.path.join(repo_dir, repo_name)

    if os.path.exists(dir_path):
        print(f"Updating {repo_name}...")
        subprocess.run(["git", "pull"], cwd=dir_path)
    else:
        print(f"Cloning {repo_name}...")
        os.makedirs(dir_path, exist_ok=True)
        subprocess.run(["git", "clone", clone_url, dir_path])


os.makedirs(repo_dir, exist_ok=True)
all_repos = get_all_repos()

with ThreadPoolExecutor(max_workers=14) as executor:
    executor.map(get_repository, all_repos)